In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns


In [20]:
df_train = pd.read_csv(r"train.csv")
df_test = pd.read_csv(r"test.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# **Assume all women survived**

In [28]:
women = df_train.loc[df_train.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived: ", rate_women)

% of women who survived:  0.7420382165605095


In [29]:
men = df_train.loc[df_train.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


# **Random forest model**

In [30]:
from sklearn.ensemble import RandomForestClassifier

y = df_train['Survived']

feature = ['Pclass','Sex','Age','SibSp','Parch']

X = pd.get_dummies(df_train[feature])
X_test = pd.get_dummies(df_test[feature])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

model.fit(X,y)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerID': df_test.PassengerId, 'Survived': predictions})

output.to_csv('submission2.csv',index=False) 

# **Logist Regression Model**

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import MinMaxScaler

In [6]:
y = df_train['Survived']

feature = ['Pclass','Sex','Age','SibSp','Parch','Embarked']

X = pd.get_dummies(df_train[feature])
X_test = pd.get_dummies(df_test[feature])

# X = df_train[feature]
# X_test = df_test[feature]


In [8]:
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()

# # X['Sex'] = label_encoder.fit_transform(X['Sex'])
# X['Embarked'] = label_encoder.fit_transform(X['Embarked'])

# # X_test['Sex'] = label_encoder.fit_transform(X_test['Sex'])
# X_test['Embarked'] = label_encoder.fit_transform(X_test['Embarked'])

KeyError: 'Embarked'

In [11]:
scaler = MinMaxScaler()

X['Age'] = scaler.fit_transform(X[['Age']])
X_test['Age'] = scaler.fit_transform(X_test[['Age']])

In [12]:
X

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,0.2750,1,0,0,1,0,0,1
1,1,0.4750,1,0,1,0,1,0,0
2,3,0.3250,0,0,1,0,0,0,1
3,1,0.4375,1,0,1,0,0,0,1
4,3,0.4375,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,0.3375,0,0,0,1,0,0,1
887,1,0.2375,0,0,1,0,0,0,1
888,3,0.0000,1,2,1,0,0,0,1
889,1,0.3250,0,0,0,1,1,0,0


In [13]:
parameters_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}


model = LogisticRegression()


# grid search 
grid_search = GridSearchCV(model, parameters_grid, cv=5)

grid_search.fit(X,y)

best_C = grid_search.best_params_['C']

In [14]:
model = LogisticRegression(C=best_C)

model.fit(X,y)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerID': df_test.PassengerId, 'Survived': predictions})

In [16]:
output.to_csv('submission_C4.csv',index=False) 